In [1]:
"""
Read VIIRS active fire geolocated fire pixels output from XXviirs_access-swath.ipynb
Author: maxwell.cook@colorado.edu
"""

# Import packages
import sys, os
import xarray as xr
import geopandas as gpd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import rasterio as rio

from datetime import datetime
from datetime import timedelta
from matplotlib import pyplot as plt
from rasterio.features import rasterize
from tqdm.notebook import tqdm

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

# Projection information
geog = 'EPSG:4326'  # Geographic projection
prj = 'EPSG:5070'  # Projected coordinate system- WGS 84 NAD83 UTM Zone 13N

# File path information
maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')
# Output directories
dataraw = os.path.join(projdir,'data/spatial/raw/VIIRS/')
datamod = os.path.join(projdir,'data/spatial/mod/VIIRS/')

# File path information
print("Ready !")

Ready !


In [2]:
# Load and tidy the fire perimeter data

In [3]:
fp = os.path.join(projdir,'data/spatial/mod/NIFC/nifc-ics_2018_to_2023-aspen.gpkg')
fires = gpd.read_file(fp)

# subset to Southern Rockies
fires = fires[fires['na_l3name'] == 'Southern Rockies']

# tidy the fire id and name columns
fires.rename(columns={'NIFC_ID': 'Fire_ID', 'NIFC_NAME': 'Fire_Name'}, inplace=True)

# tidy the date columns
fires['DISCOVERY_DATE'] = pd.to_datetime(fires['DISCOVERY_DATE'])
fires['WF_CESSATION_DATE'] = pd.to_datetime(fires['WF_CESSATION_DATE'])
fires['NIFC_ACRES'] = fires['NIFC_ACRES'].astype(float)

print(f"There are {len(fires)} with > 1% aspen cover in the Southern Rockies (2018-2023)")

There are 67 with > 1% aspen cover in the Southern Rockies (2018-2023)


In [4]:
# Add a 3km buffer to wildfire perimeters
buffer_dist = 3000  # meters
fires = fires.copy() # make a copy of the original data
fires['geometry'] = fires['geometry'].buffer(buffer_dist)
print(f"Buffered fire perimeters by {buffer_dist} meters.")

Buffered fire perimeters by 3000 meters.


In [5]:
# Create spatial points from lat/lon

In [6]:
fp = os.path.join(dataraw, 'viirs_snpp_jpss1_afd_.csv')
afds = pd.read_csv(fp).reset_index(drop=True)
afds = afds.loc[:, ~afds.columns.str.startswith('Unnamed:')]
print(f"Number of fire detections: {len(afds)}")
afds.dtypes

Number of fire detections: 1784627


longitude      float64
latitude       float64
fire_mask        int64
confidence      object
frp            float64
t4             float64
t5             float64
m13            float64
acq_date        object
acq_time         int64
daynight        object
satellite       object
short_name      object
granule_id      object
geo_id          object
sample           int64
along_scan     float64
along_track    float64
scan_angle     float64
pix_area       float64
dtype: object

In [7]:
afds['acq_date'] = pd.to_datetime(afds['acq_date']) # convert to simple date format
afds['acq_date'].head(3)

0   2019-06-27
1   2019-06-27
2   2019-06-27
Name: acq_date, dtype: datetime64[ns]

In [8]:
# get a summary of fire detection confidence
afds['confidence'].value_counts()

confidence
n    1570093
h     115501
l      99033
Name: count, dtype: int64

In [9]:
# drop low confidence detections
N= len(afds)
afds = afds[afds['confidence'] != 'l']
print(f"Dropped [{N-len(afds)}] low-confidence obs.")

Dropped [99033] low-confidence obs.


In [10]:
from shapely.geometry import Point

# convert to spatial points using pixel centroid
afds['geometry'] = [Point(xy) for xy in zip(afds.longitude, afds.latitude)]
afds_ll = gpd.GeoDataFrame(afds, geometry='geometry', crs="EPSG:4326")
afds_ll = afds_ll.to_crs("EPSG:5070")
afds_ll = afds_ll.reset_index(drop=True)
afds_ll['afdID'] = afds_ll.index # add a unique ID
print(afds_ll.head())

# save this file out.
out_fp = os.path.join(datamod,'viirs_snpp_jpss1_afd_latlon.gpkg')
afds_ll.to_file(out_fp)
print(f"\nSaved spatial points to: {out_fp}")

   longitude   latitude  fire_mask confidence        frp         t4  \
0 -95.160160  34.074640          9          h  14.550305  367.00000   
1 -95.164330  34.074383          8          n  14.550305  350.57697   
2 -95.160630  34.080082          8          n   6.089355  343.91302   
3 -95.164825  34.079823          8          n   6.089355  345.46840   
4 -95.906006  34.813230          8          n   4.297072  340.28574   

          t5       m13   acq_date  acq_time  ... short_name  \
0  295.69968  2.373925 2019-06-27      2006  ...   VJ114IMG   
1  296.18445  2.373925 2019-06-27      2006  ...   VJ114IMG   
2  296.12167  1.406672 2019-06-27      2006  ...   VJ114IMG   
3  296.05000  1.406672 2019-06-27      2006  ...   VJ114IMG   
4  296.18735  1.041121 2019-06-27      2006  ...   VJ114IMG   

                                    granule_id  \
0  VJ114IMG.A2019178.2006.002.2024029081304.nc   
1  VJ114IMG.A2019178.2006.002.2024029081304.nc   
2  VJ114IMG.A2019178.2006.002.2024029081304.

In [11]:
# join to the fire perimeters
afds_ll_fire = gpd.sjoin(afds_ll, fires, how='inner', predicate='within')
afds_ll_fire.drop(columns=['index_right'], inplace=True)

# check for duplicates
dups = afds_ll_fire[afds_ll_fire.duplicated(subset='afdID', keep=False)]
print(f"[{len(dups)}/{len(afds_ll_fire)} duplicate obs.")
dups[['afdID','Fire_Name','START_YEAR','acq_date']].head()

[8916/121271 duplicate obs.


,afdID,Fire_Name,START_YEAR,acq_date
55,55,DOE CANYON,2019,2019-06-27
55,55,WEST GUARD,2018,2019-06-27
4035,4035,CALF CANYON,2022,2018-06-28
4035,4035,SARDINAS CANYON,2018,2018-06-28
4051,4051,CALF CANYON,2022,2018-06-28


In [12]:
afds_ll_fire['acq_date'].isna().sum()

0

In [13]:
# temporal filters to handle duplicates between fires
afds_ll_fire['acq_date'] = pd.to_datetime(afds_ll_fire['acq_date'])
afds_ll_fire['acq_month'] = afds_ll_fire['acq_date'].dt.month.astype(int)
afds_ll_fire['acq_year'] = afds_ll_fire['acq_date'].dt.year.astype(int)

afds_ll_fire_ = afds_ll_fire[
    (afds_ll_fire['acq_date'] >= afds_ll_fire['DISCOVERY_DATE'] - timedelta(days=14)) &
    (afds_ll_fire['acq_date'] <= afds_ll_fire['WF_CESSATION_DATE'] + timedelta(days=14))
]

dups = afds_ll_fire_[afds_ll_fire_.duplicated(subset='afdID', keep=False)]
print(f"[{len(dups)}/{len(afds_ll_fire_)} duplicate obs.")

[0/116332 duplicate obs.


In [14]:
afds_ll_fire_.columns

Index(['longitude', 'latitude', 'fire_mask', 'confidence', 'frp', 't4', 't5',
       'm13', 'acq_date', 'acq_time', 'daynight', 'satellite', 'short_name',
       'granule_id', 'geo_id', 'sample', 'along_scan', 'along_track',
       'scan_angle', 'pix_area', 'geometry', 'afdID', 'Fire_ID', 'Fire_Name',
       'NIFC_ACRES', 'FINAL_ACRES', 'pct_aspen', 'INCIDENT_ID',
       'INCIDENT_NAME', 'START_YEAR', 'CAUSE', 'DISCOVERY_DATE',
       'DISCOVERY_DOY', 'WF_CESSATION_DATE', 'WF_CESSATION_DOY',
       'STR_DESTROYED_TOTAL', 'STR_DAMAGED_TOTAL', 'STR_THREATENED_MAX',
       'EVACUATION_REPORTED', 'PEAK_EVACUATIONS', 'WF_PEAK_AERIAL',
       'WF_PEAK_PERSONNEL', 'na_l3name', 'first_obs_date', 'last_obs_date',
       'obs_count', 'acq_month', 'acq_year'],
      dtype='object')

In [15]:
# grab a count of observations for each fire
counts = afds_ll_fire_.groupby(['Fire_ID']).size().reset_index(name='count')
afds_ll_fire_ct = pd.merge(afds_ll_fire_, counts, left_on='Fire_ID', right_on='Fire_ID', how='left')
afds_ll_fire_ct['count'].describe()

count    116332.000000
mean      11939.171303
std        8382.570471
min           2.000000
25%        4004.000000
50%        8347.000000
75%       19127.000000
max       23391.000000
Name: count, dtype: float64

In [16]:
# filter to retain fires with at least N observations
n_obs = 10
afds_ll_fire_ct = afds_ll_fire_ct[afds_ll_fire_ct['count'] >= n_obs]
print(f"There are {len(afds_ll_fire_ct['Fire_ID'].unique())} fires with >= {n_obs} obs.")

There are 54 fires with >= 10 obs.


In [17]:
# subset the fire perimeter data
fires_ = fires[fires['Fire_ID'].isin(afds_ll_fire_ct['Fire_ID'].unique())]
len(fires_)

54

In [18]:
# save this file out.
out_fp = os.path.join(datamod,'viirs_snpp_jpss1_afd_latlon_aspenfires.gpkg')
afds_ll_fire_ct.to_file(out_fp)
print(f"Saved spatial points to: {out_fp}")

Saved spatial points to: /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/VIIRS/viirs_snpp_jpss1_afd_latlon_aspenfires.gpkg


In [19]:
# Create the ground area of pixels based on swath position

In [20]:
# Define the pixel buffer function for the given width and height
def pixel_area(point, width, height):
    half_width = width / 2
    half_height = height / 2
    return box(
        point.x - half_width, point.y - half_height,
        point.x + half_width, point.y + half_height
    )

afds_pix = afds_ll_fire_ct.copy()

# Apply the buffer function with along_scan and along_track values converted to meters (*1000)
afds_pix["geometry"] = afds_pix.apply(
    lambda row: pixel_area(row["geometry"], row["along_scan"] * 1000, row["along_track"] * 1000), axis=1
)

afds_pix = afds_pix.reset_index(drop=True)
afds_pix['obs_id'] = afds_pix.index # unique ID column

afds_pix.head() # check the results

,longitude,latitude,fire_mask,confidence,frp,t4,t5,m13,acq_date,acq_time,...,WF_PEAK_AERIAL,WF_PEAK_PERSONNEL,na_l3name,first_obs_date,last_obs_date,obs_count,acq_month,acq_year,count,obs_id
0,-108.650955,37.724865,8,n,3.009492,338.37442,304.13184,1.290213,2019-06-27,2006,...,NaN,44,Southern Rockies,2019-06-23,2019-07-03,29.0,6,2019,59,0
1,-108.651790,37.728153,8,n,3.388039,335.77356,304.47110,1.634736,2019-06-27,2006,...,NaN,44,Southern Rockies,2019-06-23,2019-07-03,29.0,6,2019,59,1
2,-108.660560,37.726720,8,n,6.081448,351.61945,303.17370,1.582977,2019-06-27,2006,...,NaN,44,Southern Rockies,2019-06-23,2019-07-03,29.0,6,2019,59,2
3,-108.664955,37.726000,8,n,4.429165,341.46454,305.51712,1.840156,2019-06-27,2006,...,NaN,44,Southern Rockies,2019-06-23,2019-07-03,29.0,6,2019,59,3
4,-108.652620,37.731445,8,n,3.388039,335.03598,303.23315,1.634736,2019-06-27,2006,...,NaN,44,Southern Rockies,2019-06-23,2019-07-03,29.0,6,2019,59,4


In [21]:
# save this file out.
out_fp = os.path.join(datamod,'viirs_snpp_jpss1_afd_latlon_aspenfires_pixar.gpkg')
afds_pix.to_file(out_fp)
print(f"Saved to {out_fp}\n")

Saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/VIIRS/viirs_snpp_jpss1_afd_latlon_aspenfires_pixar.gpkg



In [22]:
len(afds_pix)

116312

In [23]:
afds_pix.columns

Index(['longitude', 'latitude', 'fire_mask', 'confidence', 'frp', 't4', 't5',
       'm13', 'acq_date', 'acq_time', 'daynight', 'satellite', 'short_name',
       'granule_id', 'geo_id', 'sample', 'along_scan', 'along_track',
       'scan_angle', 'pix_area', 'geometry', 'afdID', 'Fire_ID', 'Fire_Name',
       'NIFC_ACRES', 'FINAL_ACRES', 'pct_aspen', 'INCIDENT_ID',
       'INCIDENT_NAME', 'START_YEAR', 'CAUSE', 'DISCOVERY_DATE',
       'DISCOVERY_DOY', 'WF_CESSATION_DATE', 'WF_CESSATION_DOY',
       'STR_DESTROYED_TOTAL', 'STR_DAMAGED_TOTAL', 'STR_THREATENED_MAX',
       'EVACUATION_REPORTED', 'PEAK_EVACUATIONS', 'WF_PEAK_AERIAL',
       'WF_PEAK_PERSONNEL', 'na_l3name', 'first_obs_date', 'last_obs_date',
       'obs_count', 'acq_month', 'acq_year', 'count', 'obs_id'],
      dtype='object')

In [24]:
afds_pix['acq_datetime'] = afds_pix.apply(
    lambda row: convert_datetime(row['acq_date'], row['acq_time'], zone='America/Denver'), 
    axis=1
)
afds_pix[['acq_date','acq_time','acq_datetime','daynight']].head()

,acq_date,acq_time,acq_datetime,daynight
0,2019-06-27,2006,2019-06-27 14:06:00-06:00,Day
1,2019-06-27,2006,2019-06-27 14:06:00-06:00,Day
2,2019-06-27,2006,2019-06-27 14:06:00-06:00,Day
3,2019-06-27,2006,2019-06-27 14:06:00-06:00,Day
4,2019-06-27,2006,2019-06-27 14:06:00-06:00,Day


In [25]:
# check on nighttime datetimes
day_obs = afds_pix[afds_pix['daynight'] == 'Day']
day_obs['acq_time_mst'] = day_obs['acq_datetime'].dt.time

print(f"Minimum MST datetime for 'Day': {day_obs['acq_time_mst'].min()}")
print(f"Maximum MST datetime for 'Day': {day_obs['acq_time_mst'].max()}")
del day_obs

Minimum MST datetime for 'Day': 11:54:00
Maximum MST datetime for 'Day': 15:18:00


In [26]:
# check on nighttime datetimes
night_obs = afds_pix[afds_pix['daynight'] == 'Night']
night_obs['acq_time_mst'] = night_obs['acq_datetime'].dt.time

print(f"Minimum MST datetime for 'Night': {night_obs['acq_time_mst'].min()}")
print(f"Maximum MST datetime for 'Night': {night_obs['acq_time_mst'].max()}")
del night_obs

Minimum MST datetime for 'Night': 01:06:00
Maximum MST datetime for 'Night': 04:18:00


In [27]:
# Handle duplicate observations for AFDs

In [28]:
# Spatial overlap > 30% and the same acquisition date and time

# Find duplicates in space and time
drop_obs = set() # to store the observations we want to drop
overlap_threshold = 0.30 # spatial overlap (percent)

# group the observations by datetime
dt_groups = afds_pix.groupby(['Fire_ID','acq_datetime'])
print(f"Number of unique (Fire_ID, acq_datetime) groups: {len(dt_groups)}")

def process_group(group):
    drop_obs = set()

    # create a spatial index on the group
    sidx = group.sindex

    # loop obs. and identify spatial overlap
    for idx, obs in group.iterrows():
        if obs['afdID'] in drop_obs:
            continue  # Skip if already marked

        # Find overlapping geometries within the group
        overlap_idxs = list(sidx.intersection(obs.geometry.bounds))
        overlap_obs = group.iloc[overlap_idxs]

        for match_idx, match_obs in overlap_obs.iterrows():
            if match_obs['afdID'] == obs['afdID'] or match_obs['afdID'] in drop_obs:
                continue  # Skip self-comparisons or already processed

            # Calculate intersection and overlap ratio
            area = obs.geometry.intersection(match_obs.geometry).area
            ratio = area / obs.geometry.area

            # Check for spatial overlap > threshold
            if ratio > overlap_threshold:
                if match_obs['frp'] < obs['frp']:
                    drop_obs.add(match_obs['afdID'])
                else:
                    drop_obs.add(obs['afdID'])
                    break  # No need to check further for this observation

    return drop_obs

# Process observations grouped by Fire_ID and acq_datetime
for (fire_id, datetime), group in dt_groups:
    if len(group) > 1:  # Only process groups with potential duplicates
        drop_obs.update(process_group(group))

# apply to the AFD ground area data
print(f"Identified a total of [{len(drop_obs)}/{len(afds_pix)}] duplicate observations.")
afds_pix_ = afds_pix[~afds_pix['afdID'].isin(drop_obs)] # drop the duplicate obs.

del afds_pix
gc.collect()

Number of unique (Fire_ID, acq_datetime) groups: 3392
Identified a total of [13500/116312] duplicate observations.


0

In [29]:
# save this file out.
out_fp = os.path.join(datamod, 'viirs_snpp_jpss1_afd_latlon_aspenfires_pixar_nd.gpkg')
afds_pix_.to_file(out_fp)
print(f"Saved to:{out_fp}")

Saved to:/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/VIIRS/viirs_snpp_jpss1_afd_latlon_aspenfires_pixar_nd.gpkg


In [30]:
# save a version for GEE
afds_pix_gee = afds_pix_.copy()

# tidy the columns
afds_pix_gee.rename(columns={'DISCOVERY_DATE': 'Ig_Date', 'WF_CESSATION_DATE': 'Last_Date'}, inplace=True)
afds_pix_gee = afds_pix_gee[['afdID','acq_date','daynight','Fire_ID','Ig_Date','Last_Date','geometry']]
date_cols = ['acq_date', 'Ig_Date', 'Last_Date']
for col in date_cols:
    afds_pix_gee[col] = afds_pix_gee[col].dt.date.astype(str)
print(afds_pix_gee.dtypes)

# export shapefile
out_fp = os.path.join(projdir, 'data/earth-engine/imports/viirs_snpp_jpss1_afd_latlon_aspenfires_pixar.shp')
afds_pix_gee.to_file(out_fp)
print(f"Saved to:{out_fp}")

del afds_pix_gee
gc.collect()

afdID           int64
acq_date       object
daynight       object
Fire_ID        object
Ig_Date        object
Last_Date      object
geometry     geometry
dtype: object
Saved to:/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/earth-engine/imports/viirs_snpp_jpss1_afd_latlon_aspenfires_pixar.shp


67

In [31]:
# also save a fire perimeter dataset for GEE
fires_ = fires[fires['Fire_ID'].isin(afds_pix_['Fire_ID'])]
fires_ = fires_[['Fire_ID','Fire_Name','START_YEAR','DISCOVERY_DATE','WF_CESSATION_DATE','geometry']]
fires_.rename(columns={'START_YEAR': 'Fire_Year', 'DISCOVERY_DATE': 'Ig_Date', 'WF_CESSATION_DATE': 'Last_Date'}, inplace=True)
fires_['Ig_Date'] = fires_['Ig_Date'].dt.date.astype(str)
fires_['Last_Date'] = fires_['Last_Date'].dt.date.astype(str)

out_fp = os.path.join(projdir, 'data/earth-engine/imports/nifc_aspenfires_2018to2023.shp')
fires_.to_file(out_fp)

print(len(fires_))

print(f"\nSaved to:{out_fp}")

54

Saved to:/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/earth-engine/imports/nifc_aspenfires_2018to2023.shp


In [32]:
# create a regular grid extracted to fire perimeters
def regular_grid(extent, res=0.0035, crs_out='EPSG:5070', regions=None):
    """
    Creates a regular-spaced grid
    """
    # retrieve bounding coordinates
    min_lon, max_lon, min_lat, max_lat = extent
    
    # create the grid lines in degrees
    x_coords = np.arange(min_lon, max_lon, res)
    y_coords = np.arange(min_lat, max_lat, res)

    # generate the grid cells
    cells = [
        Polygon([(x, y), (x + res, y), (x + res, y + res), (x, y + res)])
        for x in x_coords for y in y_coords
    ]

    # create a geodataframe in WGS, reprojected if needed
    grid = gpd.GeoDataFrame({'geometry': cells}, crs=crs_out)

    if regions is not None:
        if regions.crs != grid.crs:
            regions = regions.to_crs(grid.crs)
        # Perform spatial intersection to keep only grid cells overlapping the polygon
        grid = grid[grid.intersects(regions.unary_union)].copy()

    return grid
    

# load the srm epa level III
fp = os.path.join(projdir,'data/spatial/raw/boundaries/na_cec_eco_l3_west.gpkg')
ecol3 = gpd.read_file(fp)
srm = ecol3[ecol3['NA_L3NAME'] == 'Southern Rockies']

# get the SRM extent in lat/lon (WGS)
coords, extent = get_coords(srm, buffer=3000, crs='EPSG:5070')
print(f"Bounding extent for the SRM: {extent}")

# generate the grid (0.0035 degrees or 375m)
# extract grid intersecting fire perimeters
grid = regular_grid(extent, res=375, crs_out='EPSG:5070', regions=fires_)

# save this out.
out_fp = os.path.join(datamod, 'aspenfires_rgrid_375m.gpkg')
grid.to_file(out_fp, driver="GPKG")
print(f"Grid saved to: {out_fp}")

Bounding extent for the SRM: [-1184484.4171543047, -689479.4244799014, 1397166.4988387332, 2246127.1529145916]
Grid saved to: /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/VIIRS/aspenfires_rgrid_375m.gpkg


In [33]:
# calculate a gridded cumulative FRP and maximum FRP

In [34]:
# process the fire data
t0 = time.time()

def aggregate_frp(detections, grid):
    """
    Aggregate fire pixel frp data into a regular grid with fractional overlay
    """
    
    if detections.crs != grid.crs:
        detections = detections.to_crs(grid.crs)

    # calculate the pixel area in m2
    detections['pix_area_m2'] = detections['pix_area'] * 1e6

    # tidy the grid and calculate the grid area
    grid = grid.reset_index(drop=False).rename(columns={'index': 'grid_index'})
    grid['grid_area'] = grid.geometry.area  # precompute grid cell areas

    # overlay detections onto the grid
    overlay = gpd.overlay(detections, grid, how='intersection')

    # fractional overlap area
    overlay['overlap_m2'] = overlay.geometry.area
    overlay['fraction'] = overlay['overlap_m2'] / overlay['pix_area_m2']

    # multiply FRP by fractional area
    overlay['frp_fr'] = overlay['frp'] * overlay['fraction']
    overlay['frp_fr'] = overlay['frp_fr'].fillna(0)
    
    # Aggregate by grid cell
    aggregated = overlay.groupby('grid_index').agg(
        afd_count=('frp_fr', 'count'), 
        frp_csum=('frp_fr', 'sum'),
        frp_max=('frp_fr', 'max'),
        frp_min=('frp_fr', 'min'),
        day_max_frp=('frp_fr', lambda x: overlay.loc[x.idxmax(), 'acq_date'] if not x.empty else None),
        dt_max_frp=('frp_fr', lambda x: overlay.loc[x.idxmax(), 'acq_datetime'] if not x.empty else None),
        first_obs_date=('acq_date', 'min'),
        last_obs_date=('acq_date', 'max')
    ).reset_index()

    # add day and night max FRP
    daynight_max = (
        overlay.groupby(['grid_index', 'daynight'])['frp_fr']
        .max()
        .unstack(fill_value=0)  # Ensure missing values are filled with 0
    ).reset_index()

    aggregated['frp_max_day'] = daynight_max.get('Day', 0)
    aggregated['frp_max_night'] = daynight_max.get('Night', 0)
    
    # Join results back to grid
    grid = grid.merge(aggregated, on='grid_index', how='right')
        
    return grid


# Initialize results list
fire_grids = []
for _, fire in tqdm(fires_.iterrows(), total=len(fires_), desc="Processing fires"):
    # get the geodataframe of the fire
    fire_gdf = gpd.GeoDataFrame([fire], crs=fires_.crs)  # Ensure GeoDataFrame
    
    # aggregate fire pixels to the grid
    fire_grid = aggregate_frp(afds_pix_[afds_pix_['Fire_ID'] == fire['Fire_ID']], grid)
    fire_grid['Fire_ID'] = fire['Fire_ID']
    fire_grid['Fire_Name'] = fire['Fire_Name']
    fire_grids.append(fire_grid)

# Combine all grids into one
fire_grids_ = pd.concat(fire_grids)

t3 = (time.time() - t0) / 60
print(f"\nTotal elapsed time: {t3:.2f} minutes.\n")
print("\n~~~~~~~~~~\n")
print("Done!")

Processing fires:   0%|          | 0/54 [00:00<?, ?it/s]


Total elapsed time: 0.33 minutes.


~~~~~~~~~~

Done!


In [35]:
fire_grids_['afd_count'].describe()

count    49343.000000
mean        10.647974
std          9.629420
min          1.000000
25%          4.000000
50%          8.000000
75%         15.000000
max        116.000000
Name: afd_count, dtype: float64

In [36]:
fire_grids_.sort_values(by='frp_csum', ascending=False).head(10)

,grid_index,geometry,grid_area,afd_count,frp_csum,frp_max,frp_min,day_max_frp,dt_max_frp,first_obs_date,last_obs_date,frp_max_day,frp_max_night,Fire_ID,Fire_Name
2620,1978265,"POLYGON ((-857109.417 2069541.499, -856734.417...",140625.0,6,870.865290,867.966106,0.100693,2020-09-19,2020-09-19 14:18:00-06:00,2020-09-19,2020-09-26,867.966106,0.999284,42043,MULLEN
562,1964399,"POLYGON ((-859359.417 1963791.499, -858984.417...",140625.0,14,853.682898,808.018133,0.000680,2020-10-22,2020-10-22 03:24:00-06:00,2020-10-22,2020-10-22,10.347825,808.018133,42036,EAST TROUBLESOME
4282,2023510,"POLYGON ((-849609.417 2056416.499, -849234.417...",140625.0,11,791.806956,719.896884,0.000124,2020-09-26,2020-09-26 03:12:00-06:00,2020-09-26,2020-09-27,0.000000,719.896884,42043,MULLEN
769,1973452,"POLYGON ((-857859.417 1962666.499, -857484.417...",140625.0,11,748.426058,604.198932,0.892342,2020-10-22,2020-10-22 03:24:00-06:00,2020-10-22,2020-10-22,1.140053,604.198932,42036,EAST TROUBLESOME
612,1966663,"POLYGON ((-858984.417 1963791.499, -858609.417...",140625.0,14,719.249008,396.879960,0.076479,2020-10-22,2020-10-22 03:24:00-06:00,2020-10-22,2020-10-22,10.775112,396.879960,42036,EAST TROUBLESOME
831,1999262,"POLYGON ((-853359.417 1453416.499, -852984.417...",140625.0,15,708.307866,355.249186,0.198258,2022-05-10,2022-05-10 01:54:00-06:00,2022-05-10,2022-05-11,168.903463,355.249186,13356,CALF CANYON
7194,2096707,"POLYGON ((-837234.417 1488291.499, -836859.417...",140625.0,14,697.628186,607.880565,0.049085,2022-05-10,2022-05-10 14:06:00-06:00,2022-05-10,2022-05-11,607.880565,0.689785,13356,CALF CANYON
467,1959868,"POLYGON ((-860109.417 1962666.499, -859734.417...",140625.0,6,687.762053,562.199035,1.176617,2020-10-22,2020-10-22 03:24:00-06:00,2020-10-22,2020-10-22,0.000000,562.199035,42036,EAST TROUBLESOME
4983,2057471,"POLYGON ((-843984.417 2056791.499, -843609.417...",140625.0,28,653.069204,543.823739,0.025935,2020-09-26,2020-09-26 03:12:00-06:00,2020-09-26,2020-10-04,15.932777,543.823739,42043,MULLEN
830,1999261,"POLYGON ((-853359.417 1453041.499, -852984.417...",140625.0,25,652.902505,258.996539,0.017976,2022-05-10,2022-05-10 01:54:00-06:00,2022-04-22,2022-05-11,66.215299,258.996539,13356,CALF CANYON


In [37]:
fire_grids_['frp_csum'].sum()

2475707.141178472

In [38]:
# save this file out.
out_fp = os.path.join(datamod,'viirs_snpp_jpss1_afd_latlon_aspenfires_pixar_gridstats.gpkg')
fire_grids_.to_file(out_fp)
print(f"Saved file to: {out_fp}")

Saved file to: /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/VIIRS/viirs_snpp_jpss1_afd_latlon_aspenfires_pixar_gridstats.gpkg


In [ ]:
# save a file for GEE (cleaned columns)

In [39]:
fire_grids_.columns

Index(['grid_index', 'geometry', 'grid_area', 'afd_count', 'frp_csum',
       'frp_max', 'frp_min', 'day_max_frp', 'dt_max_frp', 'first_obs_date',
       'last_obs_date', 'frp_max_day', 'frp_max_night', 'Fire_ID',
       'Fire_Name'],
      dtype='object')

In [49]:
fires_.columns

Index(['grid_index', 'Fire_ID', 'afd_count', 'max_date', 'first_date',
       'last_date', 'geometry', 'Fire_Year', 'Ig_Date', 'Last_Date'],
      dtype='object')

In [51]:
fire_grids_gee = fire_grids_.copy()

# tidy the columns
fire_grids_gee = fire_grids_gee[['grid_index','Fire_ID','afd_count','day_max_frp','first_obs_date','last_obs_date','geometry']]
fire_grids_gee.rename(columns={'day_max_frp': 'max_date', 'first_obs_date': 'first_date', 'last_obs_date': 'last_date'}, inplace=True)
    
# join in the fire ignition dates
fire_grids_gee = fire_grids_gee.merge(fires_[['Fire_ID', 'Fire_Year', 'Ig_Date', 'Last_Date']], on='Fire_ID', how='left')

# handle date fields for GEE
date_cols = ['max_date', 'first_date', 'last_date', 'Ig_Date', 'Last_Date']
for col in date_cols:
    fire_grids_gee[col] = fire_grids_gee[col].astype(str)
    
fire_grids_gee.head()

,grid_index,Fire_ID,afd_count,max_date,first_date,last_date,geometry,Fire_Year,Ig_Date,Last_Date
0,822559,14,1,2018-07-05,2018-07-05,2018-07-05,"POLYGON ((-1048359.417 1669791.499, -1047984.4...",2018,2018-06-01,2018-07-03
1,822560,14,1,2018-07-05,2018-07-05,2018-07-05,"POLYGON ((-1048359.417 1670166.499, -1047984.4...",2018,2018-06-01,2018-07-03
2,822561,14,1,2018-07-05,2018-07-05,2018-07-05,"POLYGON ((-1048359.417 1670541.499, -1047984.4...",2018,2018-06-01,2018-07-03
3,824823,14,1,2018-07-05,2018-07-05,2018-07-05,"POLYGON ((-1047984.417 1669791.499, -1047609.4...",2018,2018-06-01,2018-07-03
4,824824,14,1,2018-07-05,2018-07-05,2018-07-05,"POLYGON ((-1047984.417 1670166.499, -1047609.4...",2018,2018-06-01,2018-07-03


In [52]:
out_fp = os.path.join(projdir,'data/earth-engine/imports/viirs_snpp_jpss1_afd_latlon_aspenfires_pixar_gridstats.shp')
fire_grids_gee.to_file(out_fp)
print(f"Exported layer to: {out_fp}")

Exported layer to: /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/earth-engine/imports/viirs_snpp_jpss1_afd_latlon_aspenfires_pixar_gridstats.shp


In [53]:
gc.collect()

1864